In [1]:
import altair as alt
import csv
import numpy as np
import organize_functions
from os import listdir
import pandas as pd
import pprint
import re
import sort_bus_by_date
import seaborn as sns
from IPython.display import HTML

import matplotlib.pyplot as plt

In [2]:
directory = organize_functions.find_directory()

In [3]:
directory

'/mnt/d/Documents/Capstone/KCM-Data/Organization/Raw data/False_files_2/'

# Function for finding modules that are replaced.

In [4]:
def find_replaced_modules(directory):
    serial_index = 17
    bus_swapped_mods = {}  # Storing modules that have been swapped with each bus number
    for folder in listdir(directory):
        if folder.startswith('bus'):
            bus = folder
            bus_slash = folder + '/'
            replaced_mods = set()  # For storing modules that are confirmed swapped in and out
            modules_by_date = {}  # Storing every module per date
            serial_start_end = {}  # Storing each unique module (from set) and their starting and end times
            mod_set = set()  # A set (unordered, no repeats) of all module numbers in the bus folder
#             case of letter (i.e. to avoid A1 != a1)
            for file in listdir(directory + bus_slash):  # For each bus folder
                df = sort_bus_by_date.sort_bus_by_date(directory, bus_slash)  # Sarah's dataframe for organized files
                ordered_dates = []  # List of ordered dates per bus folder
                ordered_csv = df['Filename'].tolist()  # List of sorted CSV's by date
                ordered_unclean_dates = df['DateRetrieved'].tolist()  # List of sorted dates corresponding to CSVs
                for unclean_date in ordered_unclean_dates:
                    split_results = unclean_date.strftime('%m/%d/%Y, %H:%M:%S')
                    ordered_dates.append(split_results)  # Organized dates as strings
#                 print(ordered_dates)

                for i in range(len(ordered_csv)):
                    serials_in_csv = []
                    file_path = directory + bus_slash + ordered_csv[i]
                    with open(file_path) as file:
                        reader = csv.reader(file)
                        for row in reader:
                            for element in row:
                                if 'Mfg Data (ASCII)' in element:
                                    serial_num = element[serial_index:]
                                    mod_num = re.sub(r'\W+', '', serial_num).upper()
                                    if mod_num != '':
                                        serials_in_csv.append(mod_num)  # Serials in CSV will all be regex version
                                        mod_set.add((mod_num, serial_num))  # Adding tuple with the regex and non-regex version
                                    else:
                                        pass
                                else:
                                    pass
                    modules_by_date[ordered_dates[i]] = serials_in_csv  # Date to list of serials
                # Purpose of this section is just to get a module number associated with start and end date
#                 print(len(mod_set))
                for serial_tuple in mod_set:  # Look through every serial number in bus
                    count = 0
                    first_date = ''
                    latest_date = ''
                    start_end_list = []
                    base_serial = serial_tuple[0]  # Current serial of interest
                    
                    for each_date in ordered_dates:  # For each date (keys to current_date_serials dictionary)
                        current_date_serials = modules_by_date[each_date]  # Getting list of the serials for current CSV/date
                        for comp_serial in current_date_serials:
                            if base_serial == comp_serial:  # Compare base serial to comparison serial 
                                count += 1
                                latest_date = each_date
#                                 print("Current date: ", each_date, "\nCurrent latest date: ", latest_date)
                                if count == 1:
                                    first_date = each_date
                                    start_end_list.append(first_date)
                                else:
                                    pass
                    start_end_list.append(latest_date)
                    serial_start_end[serial_tuple[1]] = start_end_list  # Uncleaned serial -> start and end dates
                    
#                 # Now compare start and end dates for each module to the first and last date of the ordered_csvs by date list

#                 pprint.pprint(serial_start_end)  # For pretty printing dictionary
#                 print("Length of the serial dictionary: ", len(serial_start_end), "\nLength of mod set: ", len(mod_set))
                for serial_key in serial_start_end:
                    start_end = serial_start_end[serial_key]
#                     print(start_end)
# #                     print('Key : ', serial_key, "\nStart and End Dates: ", start_end, "\n")
#                     print("Start Date: ", start_end[0], "\nFirst Date: ", ordered_dates[0], "\nEnd Date: ", start_end[-1],  "\nLast Date: ", ordered_dates[-1], "\n")
                    if start_end[0] != ordered_dates[0] and start_end[-1] != ordered_dates[-1]:
                        replaced_mods.add(serial_key)
                bus_swapped_mods[bus] = list(replaced_mods)
#                 print(replaced_mods)
    return {k:v for k,v in bus_swapped_mods.items() if v}


# Function for listing the start and end dates

In [8]:
def find_start_end(directory, swapped_dict):
#     pprint.pprint(swapped_dict)
    mod_info = []
    mod_num_rows_above = 9
    mod_num_index = 7
    key_list = list(swapped_dict.keys())
    column_names = ['Bus', 'Module', 'Start date', 'End date']
    for bus_key in key_list:
        ordered_dates = []
        df = sort_bus_by_date.sort_bus_by_date(directory, bus_key + '/')
        ordered_csv = df['Filename'].tolist()
        ordered_unclean_dates = df['DateRetrieved'].tolist()
        for unclean_date in ordered_unclean_dates:
            split_results = unclean_date.strftime('%m/%d/%Y, %H:%M:%S')
            ordered_dates.append(split_results)
        serials = swapped_dict[bus_key]  # Serials for bus
#         print('\nBus: ', bus_key, '\n Serials: ', serials)
        for serial in serials:  # For swapped serial in bus
            mod_num = ''
            occurrences = 0
            start_date = ''
            end_date = ''
            serial_info = []
            for i in range(len(ordered_csv)):  # For each bus folder (key value for bus to ordered files dictionary)
                csv_rows = []
                with open(directory + bus_key + '/' + ordered_csv[i]) as file:
                    reader = csv.reader(file)
                    for row in reader:
                        csv_rows.append(row)
#                 print(len(csv_rows))
                for j in range(len(csv_rows)):
                    for element in csv_rows[j]:
                        if serial in element:
                            occurrences += 1
                            if occurrences == 1:
                                start_date = ordered_dates[i]
                            end_date = ordered_dates[i]
#                             print(csv_rows[j - mod_num_rows_above][0])
                            mod_num = csv_rows[j - mod_num_rows_above][0][mod_num_index:]
            serial_info.extend([bus_key[4:], mod_num, start_date, end_date])
#             print(serial_info)
            mod_info.append(serial_info)
#     print(mod_info)
    df = pd.DataFrame(mod_info, columns = column_names)
    return df

# Using

In [6]:
replaced_mods = find_replaced_modules(directory)
replaced_mods

{'bus_1': [' 364A1585G3REV--13G2284..........'],
 'bus_111': [' 0039-1613-0QD-402335-00710......'],
 'bus_122': [" 0039-1535-0CW-402335-00710..'..."],
 'bus_15': [' 13K2474.........................'],
 'bus_2': [' 0039-1811-00U-402335-00804......'],
 'bus_25': [' 0039-1510-06B-402335-00710......',
  ' 0039-1512-0E9-402335-00710......',
  ' 364A1585G3REVC-14K1885..........'],
 'bus_28': [" 0039-1634-04H-402335-00802..'..."],
 'bus_32': [' 364A1585G2REVA-13K2342..........'],
 'bus_36': [' 364A1585G3REVB-14E6891..........',
  ' 364A1585G3REVB-14F7320..........',
  ' 364A1585G3REVB-14E6785..........'],
 'bus_40': [' 0039-1638-0HC-402335-00802..&...'],
 'bus_58': [' 0039-1630-0GB-402335-00710......'],
 'bus_6': [' 0039-1609-009-402335-00710......'],
 'bus_77': [" 0039-1635-06T-402335-00802..'...",
  ' 0039-1434-03J-402335-00710......'],
 'bus_78': [' 0039-1632-04H-402335-00710......'],
 'bus_85': [' 0039-1639-061-402335-00802..&...'],
 'bus_87': [' 364A1585G3REVC-14K2029..........']}

In [9]:
find_start_end(directory, replaced_mods)

,Bus,Module,Start date,End date
0,1,2,"11/14/2016, 09:56:09","02/03/2017, 06:25:25"
1,111,12,"09/09/2016, 08:06:49","10/27/2017, 08:19:08"
2,122,8,"09/05/2017, 08:48:54","09/05/2017, 08:48:54"
3,15,2,"05/11/2017, 09:25:00","07/31/2017, 09:27:45"
4,2,11,"09/20/2018, 06:30:57","12/26/2018, 08:11:19"
5,25,6,"04/04/2017, 07:03:20","04/04/2017, 07:03:20"
6,25,4,"04/04/2017, 07:03:20","04/04/2017, 07:03:20"
7,25,3,"04/04/2017, 07:03:20","04/04/2017, 07:03:20"
8,28,16,"09/08/2017, 10:42:53","09/08/2017, 10:42:53"
9,32,3,"12/05/2016, 06:16:34","12/05/2016, 06:16:34"
